In [4]:
import json
import math
import pandas as pd

# Load the COCO-style JSON dataset
with open(r"C:\Users\sskin\Desktop\tflite_gauge_reader_app\dataset\train\_annotations.coco.json", "r") as f:
    data = json.load(f)

# Define default gauge parameters
DEFAULT_MIN_VALUE = 0
DEFAULT_MAX_VALUE = 100
DEFAULT_SWEEP_ANGLE = 270  # Default sweep angle (degrees)

# Function to calculate the angle of the needle
def calculate_angle(base, tip):
    x1, y1 = base
    x2, y2 = tip
    angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
    # Normalize angle to be between 0 and 360
    return angle if angle >= 0 else angle + 360

# Process images and annotations
processed_data = []
for image in data["images"]:
    image_id = image["id"]
    image_name = image["file_name"]

    # Get annotations for this image
    annotations = [ann for ann in data["annotations"] if ann["image_id"] == image_id]

    # Initialize variables for needle base, needle tip, min/max values, and sweep angle
    needle_base = None
    needle_tip = None
    min_value = DEFAULT_MIN_VALUE
    max_value = DEFAULT_MAX_VALUE
    sweep_angle = DEFAULT_SWEEP_ANGLE  # Default sweep angle

    # Extract coordinates for key components
    for ann in annotations:
        category_id = ann["category_id"]
        bbox = ann["bbox"]
        x_center = bbox[0] + bbox[2] / 2  # x + width/2
        y_center = bbox[1] + bbox[3] / 2  # y + height/2

        if data["categories"][category_id]["name"] == "Needle_base":
            needle_base = (x_center, y_center)
        elif data["categories"][category_id]["name"] == "Needle_Tip":
            needle_tip = (x_center, y_center)
        elif data["categories"][category_id]["name"] == "min_value":
            min_value = DEFAULT_MIN_VALUE  # Replace if min value is dynamic
        elif data["categories"][category_id]["name"] == "max_value":
            max_value = DEFAULT_MAX_VALUE  # Replace if max value is dynamic
        elif data["categories"][category_id]["name"] == "sweep_angle":
            sweep_angle = bbox[0]  # Assuming sweep angle is provided in a specific way (replace as needed)

    # Ensure both needle base and tip are found
    if needle_base and needle_tip:
        # Calculate the angle of the needle
        needle_angle = calculate_angle(needle_base, needle_tip)

        # Map the angle to a gauge value
        start_angle = 135  # Adjust based on gauge design (e.g., 135 for typical bottom-left start)
        normalized_angle = (needle_angle - start_angle) % 360
        gauge_value = min_value + (max_value - min_value) * (normalized_angle / sweep_angle)

        # Append processed data
        processed_data.append({
            "image_id": image_name,
            "min_value": min_value,
            "max_value": max_value,
            "sweep_angle": sweep_angle,
            "needle_angle": needle_angle,
            "gauge_value": gauge_value
        })

# Save the processed data to a CSV file
df = pd.DataFrame(processed_data)
df.to_csv("processed_gauge_train.csv", index=False)

print("Dataset processing complete! Saved as 'processed_gauge_train.csv'.")

Dataset processing complete! Saved as 'processed_gauge_train.csv'.
